# Frequency regulation — Simulating the swing equation and the effect of primary reserve (FCR)

This notebook contains a **temporal simulation of grid frequency**, assuming a simple swing equation,
at the time of **power imbalance** event (electric power consumption ≠ injection, e.g. when loosing one large generation unit).
This triggers **primary frequency regulation** (also called FCR: Frequency Containment Reserve) to stabilize the grid.

This model can show:
- effect of the absence of primary frequency regulation → grid blackout (or over-frequency)
- effect of main parameters: imbalance intensity, grid inertia, regulation gain
- effect of imperfect regulation (lag in the regulation) → oscillations

This is the “no code” notebook variant of where *formulas and simulation code are hidden* (focus only on the interactive simulation). → See 
[Frequency regulation.ipynb](Frequency%20regulation.ipynb) if you want to see how it works 🧑‍💻.

Pierre Haessig, september 2023, CC-BY

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licence Creative Commons Attribution 4.0 International" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a>

In [1]:
try: # Install ipywidgets package in JupyterLite, if run in JupyterLite
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass
import numpy, scipy, matplotlib

In [2]:
%run -i 'Frequency regulation setup.py'

---

## Interactive experiment

*Intructions: run the cell below ("Run" button with the triangle icon ▶️). Then read instructions below the graph.*

In [3]:
interact(freq_response_interact,
    ΔP_load=(-0.2, 0.2, 0.05),
    H=(1e-3, 8, 1),
    s=(1e-4, 0.10, 0.01),
    T_fcr=(1e-3, 2, 0.5),
    T_ΔP=['permanent','4 s']);

interactive(children=(FloatSlider(value=0.1, description='ΔP_load', max=0.2, min=-0.2, step=0.05), FloatSlider…

*Here is a possible wander with the interactive simulation*

### 1) Without frequency regulation
The frequency transient simulation starts without primary reserve (`fcr` off), with a full scale for frequency (0 to 50 Hz with `zoom_freq` off).
Initial frequency is at the nominal value 50 Hz.
You can explore the effect on the frequency of:

- intensity of power mismatch (excess of consumption `ΔP_load`), which can be negative in case of a excess of production
- grid inertia (`H` is seconds)
   
Both affect the rate of change of the frequency (“RoCoF”, or *slope* of the frequency trajectory). If the frequency reaches 0, this is a **blackout**(*).
And without frequency regulation, except if the power mismatch is strictly 0, it's only a matter of time becore crashing the grid (with blackout or over-frequency).

Optionnally, you can make power mismatch becomes after zero after a few seconds (`T_ΔP` → 4s). This stops the frequency change, but doesn’t make it come back to its nominal value. Make sure however to reset to the permanent mismatch (`T_ΔP` → permanent) before going on to the next section.

### 2) With frequency regulation

Then zoom closer to nominal frequency (`zoom_freq` → on) and **activate the primary frequency regulation** (`fcr` → on).
Witness the **stabilization of the grid frequency**, but *not at its nominal value*. 
You can explore the effect of:

- regulation inverse gain `s` (smaller means stronger regulation): affects the final frequency value, but not the initial rate of change (RoCoF)
- grid inertia (`H` is seconds): affects the initial rate of change (RoCoF) as without FCR, but no effect of the final value
- intensity of power mismatch (excess of consumption `ΔP_load`): affects both the final frequency value, and the initial RoCoF

Before going on, set `H` to a realistic value of about 4 to 8 seconds and `s` to 5%.

### 3) With imperfect frequency regulation (lag)

Finally, to get a realistic frequency trajectory, we should account of the slowness of the devices (often: steam or gas turbines) which contribute to the frequency regulation. **Activate the regulation lag** (`fcr_lag` → on). Witness the **frequency oscillations** before stabilization. An important value to monitor is the frequency nadir, that is the smallest transient value. It should be kept above the threshold of the automatic devices which disconnect consumers in case of low frequency (see (*) below). You can explore the *opposing* effects of:

- inertia time constant `H`: the larger, the less oscillations and thus the higher nadir
- frequency regulation lag time constant `T_fcr`: the larger (i.e. longer lag), the more oscillations and thus the lower nadir (→ risk of disconnecting some consumers)

(*) About blackout: in real grids, emergency protection would disconnect loads before reaching 0 Hz to prevent the blackout of the core network, which can be long to recover (hours to days). However, this still means blackout for consumers.